In [ ]:
from google.colab import files
import json
import pandas as pd
import os
import numpy as np
import networkx as nx
import csv

MERGING I_CODE AND N_NAME DUPLICATES

In [ ]:
#read in edge list
edge_list_df = pd.read_csv('/content/edge_list.csv') 

#read in node list 
node_list_df = pd.read_csv('/content/node_list.csv') 

In [ ]:
#create empty to-be-merged list and Source/Target checks
tbm_list = []
Source = False
Target = False

#iterate through edge list
for index, row in edge_list_sub.iterrows():
#if an edge has "I_CODE" or "N_Name" in the Neo4j "type" column,  
  if row['type'] == 'I_CODE' or row['type'] == 'N_Name':
  #see if it's "Source" value A is in the to-be-merged list already
    if any(row['Source'] in subl for subl in tbm_list): 
      Source = True
    else:
      Source = False
    #see if it's "Target" value B is in the to-be-merged list already
    if any(row['Target'] in subl for subl in tbm_list):
      Target = True
    else:
      Target = False
    #if A is in the list but not B, add B to A's entry (and vice versa)
    if Source == True and Target == False:
      for i in range(len(tbm_list)):
        if row['Source'] in tbm_list[i]:
          tbm_list[i].append(row['Target'])
    elif Source == False and Target == True:
      for i in range(len(tbm_list)):
        if row['Target'] in tbm_list[i]:
          tbm_list[i].append(row['Source'])
     #if both are in the list, merge those two entries together (if they aren't already the same)
    elif Source == True and Target == True:
      for i in range(len(tbm_list)):
        if row['Source'] in tbm_list[i]:
          sourcelist = tbm_list[i]
          sourceIndex = i
        if row['Target'] in tbm_list[i]:
          targetlist = tbm_list[i]
          targetIndex = i
      if sourceIndex != targetIndex:
        mergelist = list(set(sourcelist + targetlist))
        tbm_list.append(mergelist)
        tbm_list.pop(sourceIndex)
        tbm_list.pop(targetIndex)
    #if neither are in the list, create a new entry with A and B
    elif Source == False and Target == False:
      tbm_list.append([row['Source'], row['Target']])
    Source = False
    Target = False

In [ ]:
#make Id numbers the indices of the node list
node_list_df_id_index = node_list_df.set_index("Id", drop = False)

#create fix edge dict and merge node dfs dataframe
fix_edge_dict = {}
merge_node_dfs = pd.DataFrame()
min_id_tracker = 0 
not_in_node_list = 0

#iterate through to-be-merged list
for i in range(len(tbm_list)):
  merge_node_data = {}
  #find min id value for new merged node id
  for j in range(len(tbm_list[i])):
    if j == 0:
      min_id_tracker = tbm_list[i][j]
    else:
      min_id_tracker = min(min_id_tracker, tbm_list[i][j])
    #create a new node (append to end of node list) with the minimum identity value as its identity, append the rest of the columns into one list
    for c in list(node_list_df_id_index.columns):
      if c != "Id":
        if c in merge_node_data:
          print(c)
          merge_node_data[c] = merge_node_data[c] + ", " + str(node_list_df_id_index.at[tbm_list[i][j], c])
        else:
          merge_node_data[c] = str(node_list_df_id_index.at[tbm_list[i][j], c])
  merge_node_data["Id"] = min_id_tracker
  merge_node_df = pd.DataFrame.from_dict(merge_node_data, orient='index').T
  merge_node_dfs = merge_node_dfs.append(merge_node_df)

  #add an entry to the fix edge dict for each original node, with the original node's original id as the key and the new merged node's id as the value
  for k in range(len(tbm_list[i])):
    fix_edge_dict[tbm_list[i][k]] = min_id_tracker
    #delete all the original nodes from the node list 
    node_list_df_id_index.drop(labels = [tbm_list[i][k]], axis = 0, inplace = True)

#append all merged nodes to node list
node_list_df_id_index =node_list_df_id_index.append(merge_node_dfs)

In [ ]:
#iterate through edge list
#If an edge has a Source value and a Target value that share an entry in tbm_list, the edge is deleted. 
#If an edge has only one or the other (either a Source value present in tbm_list or a Target value present in tbm_list), 
#then that value is updated to reflect the new merged node Id using fix_edge_dict. If an edge has a Source value and a Target 
#value that are both present in tbm_list, but in different entries, then both the Source and Target values are updated to reflect 
#the new merged node Id numbers using fix_edge_dict. 

for index, row in edge_list_df.iterrows():
  #if statements eliminate case 1 (neither in tbm_list)
  #case 2 (only source in tbm_list)
  if any(row['Source'] in subl for subl in tbm_list) and not any(row['Target'] in subl for subl in tbm_list):
    edge_list_df.at[index, "Source"] = fix_edge_dict[row['Source']]
  #case 3 (only target in tbm_list)
  if any(row['Target'] in subl for subl in tbm_list) and not any(row['Source'] in subl for subl in tbm_list):
    edge_list_df.at[index, "Target"] = fix_edge_dict[row['Target']]
  #case 4 and 5 (both in tbm_list)
  if any(row['Source'] in subl for subl in tbm_list) and any(row['Target'] in subl for subl in tbm_list):
    for i in range(len(tbm_list)):
      #case 4 (both in same entry)
      if row['Source'] in tbm_list[i] and row['Target'] in tbm_list[i]:
        edge_list_df.drop([index], inplace = True)
        break
      #case 5 (both in different entries)
      if row['Source'] in tbm_list[i] and row['Target'] not in tbm_list[i]:
        edge_list_df.at[index, "Source"] = fix_edge_dict[row['Source']]
      if row['Target'] in tbm_list[i] and row['Source'] not in tbm_list[i]:
        edge_list_df.at[index, "Target"] = fix_edge_dict[row['Target']]